In [1]:
import sys
import numpy as np
import cupy as cp

sys.path.append('../')

from mandala.nodecore import Node
from mandala.nodecore import Variable
from mandala.autodiff import autodiff
from mandala.autodiff import layer
from mandala.autodiff import initializers
from mandala import cuda

In [2]:
def linear_forward(x, W, b, xp):
    y = xp.matmul(x, W.T)
    if b is not None:
        y += b
    return y


def linear_backward_W(x, gy, xp):
    gW = xp.matmul(gy.T, x)
    return gW


def linear_backward_b(b, gy, xp):
    gb = gy.sum(axis=0)
    return gb


def linear_backward_x(W, gy, xp):
    gx = np.matmul(gy, W, xp)
    return gx


class LinearFunction(autodiff.AutoDiff):

    def forward(self, xs):
        xp = cuda.get_array_module(xs)
        x, W, b = xs
        y = Node(linear_forward, [x, W, b, xp])
        return y

    def backward(self, xs, gy):
        xp = cuda.get_array_module(xs)
        x, W, b = xs
        gW = Node(linear_backward_W, [x, gy, xp])
        gx = Node(linear_backward_x, [W, gy, xp])
        if b is not None:
            gb = Node(linear_backward_x, [W, gy, xp])
        else:
            gb = None
        return gx, gW, gb


class Linear(layer.Layer):
    def __init__(self, in_ch, out_ch, nobias=False,
                 initializer=initializers.HeNormal):
        self.W = Variable(initializer((out_ch, in_ch)))
        if nobias:
            self.b = None
        else:
            self.b = Variable(numpy.zeros(out_ch, dtype=np.float32))

    def __call__(self, x):
        return LinearFunction()([x, self.W, self.b])


In [3]:
from chainer import links as L

In [4]:
L.Linear(3, 3).b.shape

(3,)

In [4]:
test = Linear(5, 5)

NameError: name 'numpy' is not defined

In [6]:
test.to_gpu()

NameError: name 'test' is not defined

In [5]:
import numpy


def HeNormal(shape, scale=1.0):
    fan_in = numpy.prod(shape[1:])
    std = scale * numpy.sqrt(2 / fan_in)
    init_W = numpy.random.normal(0, std, shape)
    return init_W.astype(numpy.float32)


In [6]:
numpy.prod((1, 3, 10, 2)[:-1])

30

In [3]:
x = np.zeros((3, 3), dtype=np.float32)

In [4]:
x = Variable(x)

In [5]:
h = x + 2

In [6]:
y = h ** 2

In [9]:
h._reference_count

0

In [10]:
y.data

array([[ 4.,  4.,  4.],
       [ 4.,  4.,  4.],
       [ 4.,  4.,  4.]], dtype=float32)

In [12]:
x._reference_count

1

In [13]:
h._reference_count

0

In [13]:
np.ndarray.__setitem__?

Signature:      np.ndarray.__setitem__(self, key, value, /)
Call signature: np.ndarray.__setitem__(*args, **kwargs)
Type:           wrapper_descriptor
String form:    <slot wrapper '__setitem__' of 'numpy.ndarray' objects>
Docstring:      Set self[key] to value.
